* http://www.greenproduct.go.kr/web/product/envProductTotal.do#fnSelectProd

In [1]:
import pandas as pd
import re

In [2]:
import requests
from bs4 import BeautifulSoup as bs

## 용도별 녹색제품

In [3]:
r_url = "http://www.greenproduct.go.kr/web/product/envProductRegionAjax.do?s_region=700711"

In [4]:
response = requests.post(r_url)
html = bs(response.json()["Content"], "lxml")

In [5]:
txt = html.select("a")[5]["onclick"].split(",")[1].replace('"', '')
re.sub("[^0-9]", "", txt)

'3692300757'

In [6]:
120-34

86

In [7]:
category = {}
prod = []
for a in html.select("a"):
    if a["href"] == "#elcsClnm":
        s = a.get_text().split()
        key = s[0].replace("[", "").replace("]", "")
        category[key] = s[1]
    else:
#         print(a.get_text())
        txt_0 = a["onclick"].split(",")[0].replace('"', '')
        txt_1 = a["onclick"].split(",")[1].replace('"', '')
        p_0 = re.sub("[^0-9]", "", txt_0)
        p_1 = re.sub("[^0-9]", "", txt_1)
        prod.append((p_0, p_1))
len(category), len(prod)

(34, 86)

## 용도별 인증기업

In [8]:
def get_html_table(prod):
    key = prod[0]
    val = prod[1]
    base_url = "http://www.greenproduct.go.kr/web/product/envProductListAjax.do"
    params = f"elcsElcs={key}&prodVcod={val}&pageNo=1&s_gubn=&s_findText="
    url = f"{base_url}?{params}"
    response = requests.post(url)
    html = bs(response.json()["Content"], "lxml")
    temp = pd.read_html(str(html))[0]
    temp["용도"] = key
    temp["제품번호"] = val
    return temp

In [9]:
prod = ("101", "5028211315")
get_html_table(prod)

,제품사진,용도명,상표명(모델명),인증업체명,전화번호,지역,G2B분류번호,G2B식별번호,용도,제품번호
0,NaN,"신문용지(기타, 54g)","참일꾼신문용지 54g/m², B4",만승자립원,053-986-3080,대구,14121999,21193595,101,5028211315
1,NaN,"신문용지(기타, 54g)","참일꾼신문용지 54g/m², A4",만승자립원,053-986-3080,대구,14121999,21193596,101,5028211315
2,NaN,"비도공지(중질지, 70g)","참일꾼중질지 70g/m², B4, 미색",만승자립원,053-986-3080,대구,14111511,22448688,101,5028211315
3,NaN,"비도공지(중질지, 70g)","참일꾼중질지 70g/m², A4, 미색",만승자립원,053-986-3080,대구,14111511,22448687,101,5028211315
4,NaN,"비도공지(중질지, 70g)","참일꾼중질지 70g/m², B4",만승자립원,053-986-3080,대구,14111511,22073250,101,5028211315
5,NaN,"비도공지(중질지, 70g)","참일꾼중질지 70g/m², A4",만승자립원,053-986-3080,대구,14111511,22073249,101,5028211315
6,NaN,"비도공지(중질지, 60g)","참일꾼중질지 60g/m², B4",만승자립원,053-986-3080,대구,14111511,21284747,101,5028211315
7,NaN,"비도공지(중질지, 60g)","참일꾼중질지 60g/m², A4",만승자립원,053-986-3080,대구,14111511,21284748,101,5028211315


In [10]:
item_list = []
for p in prod:
    df_table = get_html_table(p)
    item_list.append(df_table)

In [11]:
df = pd.concat(item_list)
del df["제품사진"]

In [12]:
df["용도"] = df["용도"].map(category)

In [13]:
df.shape

(2, 9)

In [14]:
df.to_csv("녹색제품.csv", index=False)

In [15]:
pd.read_csv("녹색제품.csv")

,용도명,상표명(모델명),인증업체명,전화번호,지역,G2B분류번호,G2B식별번호,용도,제품번호
0,검색된 결과가 없습니다.,검색된 결과가 없습니다.,검색된 결과가 없습니다.,검색된 결과가 없습니다.,검색된 결과가 없습니다.,검색된 결과가 없습니다.,검색된 결과가 없습니다.,NaN,0
1,검색된 결과가 없습니다.,검색된 결과가 없습니다.,검색된 결과가 없습니다.,검색된 결과가 없습니다.,검색된 결과가 없습니다.,검색된 결과가 없습니다.,검색된 결과가 없습니다.,NaN,0
